In [3]:
import numpy as np
import os
import shutil

def getfileslocation(file_out):
    import ntpath
    path_out = ntpath.dirname(file_out)
    return path_out

file_out="D:\D Drive\Vermilion\Surveys\From Ops\\test" #This is the location of XYZ files. should be an input in GUI
file_out2=getfileslocation(file_out)
#def USorDS:
    
start=1 #Does the data start at upstream or downstream end of reach? upstream = 0, downstream=1

def xyz2csv(file_out):
    os.chdir(file_out) #change directory to where the XYZ files are stored.
    try: #have to possibly create a secondary folder because windows permissions sometimes don't let us delete folders for about 15 minutes
        try:
            shutil.rmtree(file_out+"\CSV")
            csvfolder="CSV"
            os.mkdir(file_out+'\\'+csvfolder, 0o755)
        except:
            csvfolder="CSV"
            os.mkdir(file_out+'\\'+csvfolder, 0o755)
    except:
        try:
            shutil.rmtree(file_out+"\CSV1")
        except:
            print('.')
        csvfolder="CSV1"
        os.mkdir(file_out+'\\'+csvfolder, 0o755) #create new directory to store CSVs, so original XYZs are not overwritten

    ####Copy survey files into new directory and convert to CSV format####
    for files in os.listdir("."):
        if os.path.isfile(files) == True:
            beg = files[0:-3]
            new= (beg+"csv")
            shutil.copy(files, ".//"+csvfolder+"//"+new)
    os.chdir(file_out+ "\\" + csvfolder)
    ########################
    csvlocation=file_out+ "\\" + csvfolder
    return csvlocation
    
def combinecsv(csvlocation):
    ####Combine data from all CSVs into one CSV with data in correct order####
    filenumber = 1
    for files in os.listdir(csvlocation):
        rows=0
        data=None
        while data ==None:
            try:
                data = np.loadtxt(files,delimiter = ",", skiprows = rows, usecols=(0,1,2))
                break
            except:
                rows+=1
        firsty = data[0,1]
        lasty = data[len(data)-1,1]
        if filenumber == 1: #first survey file   
            if firsty > lasty: #first survey file, so no previous data to compare to. Assume downstream is south
                RASdata = data[::-1]
            else:
                RASdata=data
        else:    
            distfirsty = abs(previouslasty - firsty)
            distlasty = abs(previouslasty - lasty)
            if (distfirsty < distlasty): #firsty is closer, so data is in correct order
                RASdata = np.concatenate((RASdata,data))
            else: #lasty is closer, so need to reverse data
                RASdata = np.concatenate((RASdata,data[::-1]))
        previouslasty = data[len(data)-1,1] 
        filenumber+=1
    return RASdata

####THIS FUNCTION CREATES RIVER STATIONS BASED ON DISTANCE BETWEEN CROSS SECTIONS
def riverstations(RASdata):
    RASdata2 = []#[["RS","X","Y","Z"]]
    RS=1 #cross section numbering starts at 1
    for i in range(len(RASdata)):
        if i>0:
            x1=RASdata[i-1][0]
            x2=RASdata[i][0]
            y1=RASdata[i-1][1]
            y2=RASdata[i][1]
            distance=(((x2-x1)**2)+((y2-y1)**2))**(0.5)
            if distance>100:
                RS+=1
        currentpoint = [RS,RASdata[i][0],RASdata[i][1],RASdata[i][2]]
        RASdata2.append(currentpoint)
    return RASdata2
    #print(currentpoint)


####THIS FUNCTION FLIPS CROSS SECTIONS SO ALL START WITH THE LEFT SIDE, WHICH IS PREFERABLE IN HEC-RAS
def flipXS(RSXYZ):
    XS = RSXYZ
    numXS = (RSXYZ[0][0]) #number of cross sections
    XS2 = []
    XSflip = []
    flip = 0
    previousRS = numXS #cross sections are numbered 1 through numXS. So the first river station is same as numXS.
    previousX = XS[0][1]
    previousY = XS[0][2]
    
    for i in range(len(XS)):
        RS = XS[i][0]
        if RS != previousRS: #first line of new cross section. time to check if this cross section is facing correct way
            if flip ==1: #last cross section needs to be appended from XSflip
                for j in range(len(XSflip)):
                    XS2.append(XSflip[len(XSflip)-j-1][:])
                #XS2.append(XSflip[::-1])
            #else:
                #XS2.append()
            XSflip = [] #now empty XSflip for next time its needed
            X1 = XS[i][1] #first point in XS
            X2 = XS[i+1][1] #second point in XS
            Y1 = XS[i][2]
            Y2 = XS[i+1][2]
            try: #if X1=X2, slope is infinite and python will fail due to dviding by 0
                m = (Y2-Y1)/(X2-X1) #slope of XS
                inversem = -1/m #slope to dummy point for evaluation
            except: #cross section slope is vertical, so dummy slope is horizontal. so give 0 slope
                inversem = 0
            theta = np.arctan(inversem)
            dx = np.absolute(100*np.cos(theta))
            dy = np.absolute(100*np.sin(theta))
            if Y2>Y1:
                if X2>X1:
                    X3 = X2 - dx #dummy point for evaluation
                    Y3 = Y2 + dy
                else:
                    X3 = X2 - dx #dummy point for evaluation
                    Y3 = Y2 - dy
            else:
                if X2>X1:    
                    X3 = X2 + dx #dummy point for evaluation
                    Y3 = Y2 + dy
                else:
                    X3 = X2 + dx #dummy point for evaluation
                    Y3 = Y2 - dy
            distreal = (((X2-previousX)**2)+((Y2-previousY))**2)**0.5 #distance from real cross section to the next upstream XS
            distdummy = (((X3-previousX)**2)+((Y3-previousY))**2)**0.5 #distance from dummy XS (downstream of real) to next upstream XS
            if distdummy > distreal: #cross section is correct
                #print(RS, "is good!")
                flip = 0
            else: #cross section needs to be flipped
                #print (RS, "needs to be flipped!")
                flip= 1
            previousRS = RS #keeping current river station so don't come into this river station again
            previousX = XS[i][1] #keeping current cross section's first coordinate for comparison to next cross section
            previousY = XS[i][2]
        if flip == 1:
            XSflip.append(XS[i][:])
        else:
            XS2.append(XS[i][:])
    if flip ==1: #last cross section needs to be appended from XSflip
        for j in range(len(XSflip)):
            XS2.append(XSflip[len(XSflip)-j-1][:])
    return XS2

####FOR TESTING#####
#RASdata=RASdata[::-1][:]

csvlocation=xyz2csv(file_out)
RASdata=combinecsv(csvlocation)
if start == 0:
    RASdata=RASdata[::-1][:]

RASdata2=riverstations(RASdata)
RASdata2=np.array(RASdata2)
RASdata3=flipXS(RASdata2[::-1][:])
RASdata3=np.array(RASdata3)
np.savetxt(csvlocation+'\RASdata3.csv',RASdata3, delimiter = ',',fmt='%s')  

In [ ]:
#### NEED A FUNCTION THAT CAN SEE IF A CROSS SECTION IS NUMBERED CORRECTLY
def reorderXS(RSXYZ):
    RS=RSXYZ[0][0]
    reorder=[]
    reorder.append(RS)
    averagex=0
    averagey=0
    currentx=0
    currenty=0
    currentcount=0
    count=0
    currentdistance=100000
    outputdata=[]
    for k in range(len(RSXYZ)):
        if RSXYZ[k][0]==RS: #initializing output file with first River Station
            outputdata.append(RSXYZ[k])
        else:
            comparingRS=outputdata[0][0] #initializing comparingRS with first RS
            break

    for l in range(1,int(max(RSXYZ[:,0]))):#cycle through each cross section
        print(l)
        comparingx=0
        comparingy=0
        comparingcount=0
        for j in range(len(outputdata)):
            if outputdata[j][0]==comparingRS:
                comparingx=comparingx+outputdata[j][1]
                comparingy=comparingy+outputdata[j][2]
                comparingcount+=1
        
        
        
        
        
        
        
    for i in range(len(RSXYZ)):#go through each River Station from beginning to end
        if RS== RSXYZ[i][0]:#still cycling through "current comparing" cross section, so keep adding
            currentx=currentx+RSXYZ[i][1]
            currenty=currenty+RSXYZ[i][2]
            currentcount+=1
        else:#done with "current comparing" cross section. Now look at potential neighbors
            print("current comparing RS is ",RS)
            nextRS=RSXYZ[i][0]
            for j in range((i),len(RSXYZ)): #now check every cross section that has not yet been added, to find closest
                if nextRS == RSXYZ[j][0]:#still in same cross section, keep adding
                    averagex=averagex+RSXYZ[j][1]
                    averagey=averagey+RSXYZ[j][2]
                    count+=1
                else:
                    print(nextRS)
                    newdistance=((((averagex/count)-(currentx/currentcount))**2)+(((averagey/count)-(currenty/currentcount))**2))**0.5
                    if newdistance < currentdistance:
                        currentdistance = newdistance
                        print("potential RS is ",nextRS)
                        potentialRS=nextRS
                    averagex=0
                    averagey=0
                    count=0
                    nextRS=RSXYZ[j][0]
            reorder.append(potentialRS)
            count=0
            currentcount=0
            averagex=0
            averagey=0
            currentx=0
            currenty=0
            currentdistance=100000
            RS=RSXYZ[i][0]
            print(RS)
    outputdata=np.array(outputdata)
    print(outputdata)
    return reorder

In [ ]:
a=reorderXS(RASdata2)

In [ ]:
file_out2

In [ ]:
print(546453445/100)

In [ ]:
averagex=1
count=1
currentx=2
currentcount=2
averagey=3
currenty=4
newdistance=(((averagex/count)-(currentx/currentcount)**2)+(((averagey/count)-(currenty/currentcount)**2)))**0.5
print(newdistance)

In [ ]:
currentx/currentcount

In [ ]:
#### NEED A FUNCTION THAT CAN SEE IF A CROSS SECTION IS NUMBERED CORRECTLY
def reorderXS(RSXYZ):
    RS=RSXYZ[0][0]
    reorder=[]
    reorder.append(RS)
    averagex=0
    averagey=0
    currentx=0
    currenty=0
    currentcount=0
    count=0
    currentdistance=100000
    outputdata=[]  
    for i in range(len(RSXYZ)):#go through each River Station from beginning to end
        if RS== RSXYZ[i][0]:#still cycling through "current comparing" cross section, so keep adding
            currentx=currentx+RSXYZ[i][1]
            currenty=currenty+RSXYZ[i][2]
            currentcount+=1
        else:#done with "current comparing" cross section. Now look at potential neighbors
            print("current comparing RS is ",RS)
            nextRS=RSXYZ[i][0]
            for j in range((i),len(RSXYZ)): #now check every cross section that has not yet been added, to find closest
                if nextRS == RSXYZ[j][0]:#still in same cross section, keep adding
                    averagex=averagex+RSXYZ[j][1]
                    averagey=averagey+RSXYZ[j][2]
                    count+=1
                else:
                    print(nextRS)
                    newdistance=((((averagex/count)-(currentx/currentcount))**2)+(((averagey/count)-(currenty/currentcount))**2))**0.5
                    if newdistance < currentdistance:
                        currentdistance = newdistance
                        print("potential RS is ",nextRS)
                        potentialRS=nextRS
                    averagex=0
                    averagey=0
                    count=0
                    nextRS=RSXYZ[j][0]
            reorder.append(potentialRS)
            count=0
            currentcount=0
            averagex=0
            averagey=0
            currentx=0
            currenty=0
            currentdistance=100000
            RS=RSXYZ[i][0]
            print(RS)
    outputdata=np.array(outputdata)
    print(outputdata)
    return reorder